In [2]:
import pandas as pd
import sklearn
from matplotlib import pyplot as plt

In [3]:
from itertools import combinations

cell_types = [
    "MesLHX9", "MesGATA2", "Supp", "preGran", "Missing", "Endo",
    "PV", "CoelEpi", "Epi", "Neuro", "Immune", "Germ", "SMC"
]

all_subsets = []
for r in range(1, len(cell_types) + 1):
    all_subsets.extend(combinations(cell_types, r))

print(f"Total subsets: {len(all_subsets)}")
# Optional: to print them
# for ind, subset in enumerate(all_subsets):
#     if ind < 10000:
#         print(subset)


Total subsets: 8191


# Row-wise


In [6]:
# filepath = "/group/tran3/gchahal/other_tissues/CardiacNetworkComponentPredictor/Duy_CardiacNetworkComponentPredictor/out/q75_week_7_9/ovary.bed"
filepath = "/group/tran3/gchahal/other_tissues/CardiacNetworkComponentPredictor/Duy_CardiacNetworkComponentPredictor/out/q75_ovary_testes_from_week_7_to_9/ovary.bed"

bed_df = pd.read_csv(filepath, sep="\t", header=None)

# Optionally assign column names (if known)
bed_df.columns = ["chrom", "start", "end"] + [f"col{i}" for i in range(4, bed_df.shape[1] + 1)]

print(bed_df.head())
print(bed_df.shape)


# score_filepath = "/group/tran3/gchahal/other_tissues/CardiacNetworkComponentPredictor/Duy_CardiacNetworkComponentPredictor/data/scATAc_data_other_Domke_cellatlas/scores_q75_week_7_9_ovary_bed_file.bed"
score_filepath = "/group/tran3/gchahal/other_tissues/CardiacNetworkComponentPredictor/Duy_CardiacNetworkComponentPredictor/data/scATAc_data_other_Domke_cellatlas/scores_q75_filtered_week_8.6_8.8_9_female_bed_file.bed"
score_df = pd.read_csv(score_filepath, sep="\t", header=None)
# Optionally assign column names (if known)
score_df.columns = ["chrom", "start", "end"] + [f"col{i}" for i in range(4, bed_df.shape[1] + 1)]
print(score_df.shape)
score_df.head()


merged_df = score_df.merge(
    bed_df[['chrom', 'start', 'end']],
    on=['chrom', 'start', 'end'],
    how='inner'
)

print(merged_df.shape)
print(merged_df.head(100))

  chrom    start      end                                               col4
0  chr1   874955   877774  ovary_MesLHX9_Supp_MesGATA2_Endo_PV_preGran_Co...
1  chr1  1249607  1251159  ovary_MesLHX9_Supp_MesGATA2_preGran_Endo_PV_Co...
2  chr1  1315047  1316174  ovary_MesGATA2_Supp_preGran_CoelEpi_MesLHX9_Mi...
3  chr1  1461877  1462903  ovary_Supp_MesLHX9_MesGATA2_preGran_Endo_Missi...
4  chr1  1550123  1552317  ovary_preGran_MesLHX9_CoelEpi_MesGATA2_Endo_Mi...
(7602, 4)
(50735, 4)
(7602, 4)
   chrom     start       end  \
0   chr1    874955    877774   
1   chr1   1249607   1251159   
2   chr1   1315047   1316174   
3   chr1   1461877   1462903   
4   chr1   1550123   1552317   
..   ...       ...       ...   
95  chr1  11847482  11848703   
96  chr1  11898095  11899535   
97  chr1  11912331  11913849   
98  chr1  11966171  11967657   
99  chr1  12143268  12144521   

                                                 col4  
0   Supp_47_MesLHX9_26_Epi_3_Endo_4_Neuro_5_Missin...  
1   Supp_5

In [7]:
# import pandas as pd
# import re

# # Suppose your dataframe is called df
df = merged_df.copy()
import pandas as pd
import re

# Example dataframe: df

def extract_counts(col4_value):
    matches = re.findall(r'(\w+?)_(\d+)', col4_value)
    count_dict = {}
    for cell_type, count in matches:
        # Remove leading underscores
        cleaned_name = cell_type.lstrip('_')
        if cleaned_name in count_dict:
            count_dict[cleaned_name] += int(count)
        else:
            count_dict[cleaned_name] = int(count)
    return count_dict

# Create per-cell-type columns
cell_counts_df = df['col4'].apply(extract_counts).apply(pd.Series)

# Fill missing values with 0
cell_counts_df = cell_counts_df.fillna(0)

# Merge back to original df
df = pd.concat([df, cell_counts_df], axis=1)

# Find dominant cell type and corresponding count
df['dominant_cell_type'] = cell_counts_df.idxmax(axis=1)
df['dominant_count'] = cell_counts_df.max(axis=1)
print(df['dominant_cell_type'].value_counts())
df



# def get_dominant_info(col4_value):
#     # Find all cell type-count pairs
#     matches = re.findall(r'(\w+?)_(\d+)', col4_value)
#     if matches:
#         # List of (cell_type, count)
#         cell_counts = [(cell_type, int(count)) for cell_type, count in matches]
#         # Find cell type with max count
#         dominant = max(cell_counts, key=lambda x: x[1])
#         return pd.Series([dominant[0], dominant[1]])
#     else:
#         return pd.Series([None, None])

# df[['dominant_cell_type', 'dominant_count']] = df['col4'].apply(get_dominant_info)

# print(df['dominant_cell_type'].value_counts())

dominant_cell_type
MesGATA2    4639
preGran     1902
Supp         883
MesLHX9       89
Neuro         60
CoelEpi       28
Endo           1
Name: count, dtype: int64


,chrom,start,end,col4,Supp,MesLHX9,Epi,Endo,Neuro,Missing,MesGATA2,preGran,PV,CoelEpi,Immune,Germ,SMC,dominant_cell_type,dominant_count
0,chr1,874955,877774,Supp_47_MesLHX9_26_Epi_3_Endo_4_Neuro_5_Missin...,47.0,26.0,3.0,4.0,5.0,8.0,98.0,35.0,6.0,21.0,1.0,0.0,0.0,MesGATA2,98.0
1,chr1,1249607,1251159,Supp_50_MesLHX9_16_Epi_3_Endo_2_Neuro_6_Missin...,50.0,16.0,3.0,2.0,6.0,7.0,37.0,68.0,2.0,13.0,1.0,0.0,0.0,preGran,68.0
2,chr1,1315047,1316174,Supp_35_MesLHX9_8_Epi_4_Neuro_3_Missing_10_Mes...,35.0,8.0,4.0,0.0,3.0,10.0,58.0,25.0,2.0,20.0,0.0,1.0,1.0,MesGATA2,58.0
3,chr1,1461877,1462903,Supp_33_MesLHX9_20_Epi_5_Endo_4_Neuro_6_Missin...,33.0,20.0,5.0,4.0,6.0,4.0,85.0,37.0,2.0,9.0,1.0,0.0,1.0,MesGATA2,85.0
4,chr1,1550123,1552317,Supp_31_MesLHX9_18_Epi_6_Endo_4_Neuro_11_Missi...,31.0,18.0,6.0,4.0,11.0,14.0,62.0,56.0,7.0,14.0,1.0,1.0,1.0,MesGATA2,62.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7597,chrX,153981033,153982212,Supp_23_MesLHX9_24_Epi_1_Endo_2_Neuro_10_Missi...,23.0,24.0,1.0,2.0,10.0,9.0,70.0,6.0,5.0,5.0,3.0,0.0,0.0,MesGATA2,70.0
7598,chrX,154093877,154095497,Supp_34_MesLHX9_9_Epi_7_Endo_3_Neuro_8_Missing...,34.0,9.0,7.0,3.0,8.0,7.0,38.0,28.0,4.0,25.0,0.0,0.0,1.0,MesGATA2,38.0
7599,chrX,154347183,154351943,Supp_48_MesLHX9_30_Epi_5_Endo_6_Neuro_7_Missin...,48.0,30.0,5.0,6.0,7.0,12.0,102.0,47.0,9.0,20.0,2.0,1.0,0.0,MesGATA2,102.0
7600,chrX,154353245,154358970,Supp_49_MesLHX9_21_Epi_8_Endo_5_Neuro_12_Missi...,49.0,21.0,8.0,5.0,12.0,13.0,61.0,62.0,7.0,18.0,2.0,2.0,1.0,preGran,62.0


In [8]:
import pandas as pd

# List of cell types you provided
dominant_cell_types = [
    "MesGATA2",
    "preGran",
    "Supp",
    "MesLHX9",
    "Neuro",
    "CoelEpi",
    "Endo"
]

for cell_type in dominant_cell_types:
    # Filter rows where dominant_cell_type matches
    df_subset = df[df['dominant_cell_type'] == cell_type][['chrom', 'start', 'end']]
    
    # Construct output file name without threshold
    output_file = f"/group/tran3/duytran/cre_pipeline/Results/cell_type_specific_regions/row_wise/ovary/ovary_{cell_type}_row_wise.bed"
    
    # Save as BED file (tab-separated, no header, no index)
    df_subset.to_csv(output_file, sep='\t', header=False, index=False)
    
    print(f"Saved {len(df_subset)} regions to {output_file}")


Saved 4639 regions to /group/tran3/duytran/cre_pipeline/Results/cell_type_specific_regions/row_wise/ovary/ovary_MesGATA2_row_wise.bed
Saved 1902 regions to /group/tran3/duytran/cre_pipeline/Results/cell_type_specific_regions/row_wise/ovary/ovary_preGran_row_wise.bed
Saved 883 regions to /group/tran3/duytran/cre_pipeline/Results/cell_type_specific_regions/row_wise/ovary/ovary_Supp_row_wise.bed
Saved 89 regions to /group/tran3/duytran/cre_pipeline/Results/cell_type_specific_regions/row_wise/ovary/ovary_MesLHX9_row_wise.bed
Saved 60 regions to /group/tran3/duytran/cre_pipeline/Results/cell_type_specific_regions/row_wise/ovary/ovary_Neuro_row_wise.bed
Saved 28 regions to /group/tran3/duytran/cre_pipeline/Results/cell_type_specific_regions/row_wise/ovary/ovary_CoelEpi_row_wise.bed
Saved 1 regions to /group/tran3/duytran/cre_pipeline/Results/cell_type_specific_regions/row_wise/ovary/ovary_Endo_row_wise.bed


In [9]:
filepath = "/group/tran3/gchahal/other_tissues/CardiacNetworkComponentPredictor/Duy_CardiacNetworkComponentPredictor/out/q75_week_7_9/ovary.bed"
bed_df = pd.read_csv(filepath, sep="\t", header=None)

# Optionally assign column names (if known)
bed_df.columns = ["chrom", "start", "end"] + [f"col{i}" for i in range(4, bed_df.shape[1] + 1)]

print(bed_df.head())
print(bed_df.shape)

  chrom    start      end                                               col4
0  chr1   874955   877774  ovary_MesLHX9_Supp_MesGATA2_Endo_PV_preGran_Co...
1  chr1  1249607  1251159  ovary_MesLHX9_Supp_MesGATA2_preGran_Endo_PV_Co...
2  chr1  1315047  1316174  ovary_MesGATA2_Supp_preGran_CoelEpi_MesLHX9_Mi...
3  chr1  1461877  1462903  ovary_Supp_MesLHX9_MesGATA2_preGran_Endo_Missi...
4  chr1  1550123  1552317  ovary_preGran_MesLHX9_CoelEpi_MesGATA2_Endo_Mi...
(7602, 4)


# Draft 

In [7]:
# Define cell types
cell_types = [
    "MesLHX9", "MesGATA2", "Supp", "preGran", "Missing", "Endo",
    "PV", "CoelEpi", "Epi", "Neuro", "Immune", "Germ", "SMC"
]

# Generate all non-empty subsets
all_subsets = []
for r in range(1, len(cell_types) + 1):
    all_subsets.extend(combinations(cell_types, r))

# Evaluate subsets across records
results = []
col4_values = bed_df["col4"].astype(str)

for subset in all_subsets:
    subset_len = len(subset)
    matched_indices = [
        idx for idx, val in enumerate(col4_values)
        if all(cell in val for cell in subset)
    ]
    results.append({
        "subset": subset,
        "subset_length": subset_len,
        "row_index": matched_indices,
        "match_count": len(matched_indices)
    })

# Convert to DataFrame
subset_df = pd.DataFrame(results)

# Example output
subset_df

,subset,subset_length,row_index,match_count
0,"(MesLHX9,)",1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",7592
1,"(MesGATA2,)",1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",7600
2,"(Supp,)",1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",7597
3,"(preGran,)",1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",7590
4,"(Missing,)",1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",7572
...,...,...,...,...
8186,"(MesLHX9, MesGATA2, Supp, Missing, Endo, PV, C...",12,"[4, 9, 57, 76, 113, 115, 149, 200, 250, 257, 2...",160
8187,"(MesLHX9, MesGATA2, preGran, Missing, Endo, PV...",12,"[4, 9, 57, 76, 113, 115, 149, 200, 250, 257, 2...",160
8188,"(MesLHX9, Supp, preGran, Missing, Endo, PV, Co...",12,"[4, 9, 57, 76, 113, 115, 149, 200, 250, 257, 2...",160
8189,"(MesGATA2, Supp, preGran, Missing, Endo, PV, C...",12,"[4, 9, 57, 76, 113, 115, 149, 200, 250, 257, 2...",160


In [8]:
subset_df_sorted = subset_df.sort_values(by=["subset_length", "match_count"], ascending=[True, False]).reset_index(drop=True)
subset_df_sorted

# MesGATA2: only specific to MesGata2 
# bed file:
# chrX - start - end - ovary_MesGATA2

# MesGATA2 + Supp 
# chrX - start - end - ovary_MesGATA2 + Supp 

# Query: 
# for subset_length between (1,13):
# explore any possible combination with this subset_length:
# 

# threshold

,subset,subset_length,row_index,match_count
0,"(MesGATA2,)",1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",7600
1,"(Supp,)",1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",7597
2,"(MesLHX9,)",1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",7592
3,"(preGran,)",1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",7590
4,"(Epi,)",1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",7590
...,...,...,...,...
8186,"(MesLHX9, MesGATA2, Supp, Missing, Endo, PV, C...",12,"[4, 9, 57, 76, 113, 115, 149, 200, 250, 257, 2...",160
8187,"(MesLHX9, MesGATA2, preGran, Missing, Endo, PV...",12,"[4, 9, 57, 76, 113, 115, 149, 200, 250, 257, 2...",160
8188,"(MesLHX9, Supp, preGran, Missing, Endo, PV, Co...",12,"[4, 9, 57, 76, 113, 115, 149, 200, 250, 257, 2...",160
8189,"(MesGATA2, Supp, preGran, Missing, Endo, PV, C...",12,"[4, 9, 57, 76, 113, 115, 149, 200, 250, 257, 2...",160


In [10]:
subset_df_sorted.tail(30)

,subset,subset_length,row_index,match_count
8161,"(MesLHX9, MesGATA2, preGran, Missing, Endo, PV...",11,"[4, 9, 57, 76, 113, 115, 149, 200, 250, 257, 2...",160
8162,"(MesLHX9, MesGATA2, preGran, Endo, PV, CoelEpi...",11,"[4, 9, 57, 76, 113, 115, 149, 200, 250, 257, 2...",160
8163,"(MesLHX9, MesGATA2, Missing, Endo, PV, CoelEpi...",11,"[4, 9, 57, 76, 113, 115, 149, 200, 250, 257, 2...",160
8164,"(MesLHX9, Supp, preGran, Missing, Endo, PV, Co...",11,"[4, 9, 57, 76, 113, 115, 149, 200, 250, 257, 2...",160
8165,"(MesLHX9, Supp, preGran, Missing, Endo, PV, Co...",11,"[4, 9, 57, 76, 113, 115, 149, 200, 250, 257, 2...",160
8166,"(MesLHX9, Supp, preGran, Missing, Endo, PV, Ep...",11,"[4, 9, 57, 76, 113, 115, 149, 200, 250, 257, 2...",160
8167,"(MesLHX9, Supp, preGran, Endo, PV, CoelEpi, Ep...",11,"[4, 9, 57, 76, 113, 115, 149, 200, 250, 257, 2...",160
8168,"(MesLHX9, Supp, Missing, Endo, PV, CoelEpi, Ep...",11,"[4, 9, 57, 76, 113, 115, 149, 200, 250, 257, 2...",160
8169,"(MesLHX9, preGran, Missing, Endo, PV, CoelEpi,...",11,"[4, 9, 57, 76, 113, 115, 149, 200, 250, 257, 2...",160
8170,"(MesGATA2, Supp, preGran, Missing, Endo, PV, C...",11,"[4, 9, 57, 76, 113, 115, 149, 200, 250, 257, 2...",160
